# Librairies

In [4]:
#Importation des librairies utilisées
import unicodedata 
import time
import pandas as pd
import pickle
import numpy as np
import random
import nltk
import collections
import itertools
import csv
import warnings
import gensim
from sklearn.cross_validation import train_test_split
import os
import nltk # Nettoyage des données

## Fonction de setmming de stemming permettant la racinisation
stemmer=nltk.stem.SnowballStemmer('french')

DATA_DIR = "/Users/bguillouet/Insa/TP_Insa/data/cdiscount_NLP/"
DATA_WORD2VEC_DIR = "/Users/bguillouet/Insa/TP_Insa/data/cdiscount_NLP/word2vec/"


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Download Data

In [5]:
data_train_clean = pd.read_csv(DATA_DIR + "Categorie_reduit_train_clean.csv").fillna("")
X_train_clean = [k.split(" ") for k in data_train_clean["Description"].values]

data_train_clean_stem = pd.read_csv(DATA_DIR + "Categorie_reduit_train_clean_stem.csv").fillna("")
X_train_clean_stem = [k.split(" ") for k in data_train_clean_stem["Description"].values]

data_valid_clean = pd.read_csv(DATA_DIR + "Categorie_reduit_valid_clean.csv").fillna("")
X_valid_clean = [k.split(" ") for k in data_valid_clean["Description"].values]

data_valid_clean_stem = pd.read_csv(DATA_DIR + "Categorie_reduit_valid_clean_stem.csv").fillna("")
X_valid_clean_stem = [k.split(" ") for k in data_valid_clean_stem["Description"].values]

In [6]:
X_train_clean

[['fermeture',
  'boutonnee',
  'maille',
  'fine',
  'toucher',
  'doux',
  'bords',
  'coteles',
  'cachemire',
  'soie',
  'viscose',
  'elasthanne',
  'laver'],
 ['coque',
  'souple',
  'rose',
  'htc',
  'one',
  'mini',
  'motif',
  'love',
  'bangkok',
  'films',
  'coque',
  'souple',
  'ultra',
  'fine',
  'rose',
  'originale',
  'muzzano',
  'motif',
  'love',
  'bangkok',
  'htc',
  'one',
  'mini',
  'films',
  'voir',
  'presentation'],
 ['coque',
  'souple',
  'grise',
  'wiko',
  'stairway',
  'motif',
  'drapeau',
  'israel',
  'coque',
  'souple',
  'ultra',
  'fine',
  'grise',
  'originale',
  'muzzano',
  'motif',
  'drapeau',
  'israel',
  'wiko',
  'stairway',
  'design',
  'sophistiq',
  'voir',
  'presentation'],
 ['samsung',
  'galaxy',
  'batterie',
  'haute',
  'densite',
  'origine',
  'samsung',
  'samsung',
  'galaxy',
  'batterie',
  'but',
  'remplacer',
  'mobile',
  'voir',
  'presentation'],
 ['batterie',
  'sony',
  'vaio',
  'vgn',
  'ion',
  'mah'

# Build Model

In [1]:
Features_dimension = 300
N_train = 900000

#sg (int {1, 0}) – Defines the training algorithm. If 1, skip-gram is employed; otherwise, CBOW is used.


#hs (int {1,0}) – If 1, hierarchical softmax will be used for model training. 
#If set to 0, and negative is non-zero, negative sampling will be used.

#negative (int) – If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). 
#If set to 0, no negative sampling is used



In [2]:
X_train

NameError: name 'X_train' is not defined

## On Clean Stem Data

In [3]:
for data_type, X in [["clean", X_train_clean], ["clean_stem", X_train_clean_stem]]:
    for sg in [0,1]:
        for hs, negative in [[1,0], [0,10]]:

            #directions_file
            
            model_clean_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_model" %(Features_dimension, sg, hs, negative)
            metadata_clean_dir  = DATA_WORD2VEC_DIR+"_"+data_type+"_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_metadata" %(Features_dimension, sg, hs, negative)
            
            is_model_clean_built = os.path.isfile(model_clean_dir)
            
            if is_model_clean_built:
                print("Model aready built")
                model = gensim.models.Word2Vec.load(model_clean_dir)
                metadata = pickle.load(open(metadata_clean_dir,"rb"))
                t_learning = metadata["learning_time"]
                N_vocab = metadata["vocab_size"]

                print("Model downloaded, vector size = %d, vocab size = %d" %model.wv.vectors.shape)
                print("Learning time = %d secondes" % t_learning)

            else:
                # Learn word2 vec emmebding on train data
                print("Start learning Word2Vec learning for "+data_type)
                print("Params are : Fdim_%d_sg_%d_hs_%d_negative_%d_model" %(Features_dimension, sg, hs, negative))
                ts = time.time()
                model = gensim.models.Word2Vec(X, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
                te = time.time()
                t_learning = te-ts

                # Metadata
                N_vocab, feature_dim = model.wv.vectors.shape
                print("Model Learn in %d seconds for %d lines, %d features spaces dimension and a total of %d words" %(t_learning, N_train, Features_dimension, N_vocab))
                metadata = {"learning_time" : t_learning, "vocab_size" : N_vocab, "sg" : sg, "data_type":data_type, "negative": negative, "hs":hs}
                pickle.dump(metadata, open(metadata_clean_dir,"wb"))
                print(metadata)

                #Save word2vec model
                model.save(model_clean_dir)
                print("Model Saved")


NameError: name 'X_train_clean' is not defined

## Somme exemple of similarity

In [5]:
columns=[]
model_dic = {}
for data_type, data in [["clean", data_train_clean], ["clean_stem", data_train_clean_stem]]:
    # Build list of token from description field
    for sg in [0,1]:
        for hs, negative in [[0,10]]:           
            #directions_file
            model_clean_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_model" %(Features_dimension, sg, hs, negative)
            column = data_type+" skip_gram" if sg==1 else data_type+ " CBOW"
            columns.append(column)
            model = gensim.models.Word2Vec.load(model_clean_dir)
            model_dic.update({column:[model, model_clean_dir]})
columns.append("online model")
model_online_dir = "/Users/bguillouet/Insa/TP_Insa/data/cdiscount_NLP/word2vec/fr/fr.bin"
model = gensim.models.Word2Vec.load(model_online_dir)
model_dic.update({"online model" : [model, model_online_dir]})

In [6]:
term="homme"

df_=[]
for column in columns:
    model = model_dic[column][0]
    token = stemmer.stem(term) if "clean_stem" in column else term
    mpow = model.predict_output_word([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])

print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : homme


,clean CBOW,clean skip_gram,clean_stem CBOW,clean_stem skip_gram,online model
0,ditla,burlington,ditl,stuhrling,d'affaires
1,boxer,lifestyles,hood,burlington,vieil
2,genre,yema,boardshort,yem,d’affaires
3,sportwear,stuhrling,bonavit,montr,politique
4,ritchie,noirgrammage,sportwear,lm_,d'état
5,tshirt,metron,montr,rerock,d'église
6,montre,stared,montres,lallur,cet
7,tungstene,montre,seiko,torgoen,jeune
8,sbrn,rerock,penfield,tissot,néandertal
9,grinders,japrag,genr,metron,d’état


In [7]:
term="femme"

df_=[]
for column in columns:
    model = model_dic[column][0]
    token = stemmer.stem(term) if "clean_stem" in column else term
    mpow = model.predict_output_word([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])

print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : femme


,clean CBOW,clean skip_gram,clean_stem CBOW,clean_stem skip_gram,online model
0,ditla,geneva,ditl,cendriyon,sa
1,vetement,cendriyon,keral,montr,jeune
2,watx,burgi,venc,genev,d'affaires
3,genre,montre,montr,tko,mariée
4,montre,brillantin,genr,parn,enceinte
5,alhajas,breda,maxazri,slw,nue
6,fatale,maxazria,slw,ubn,adultère
7,venca,destellos,fiance,akribos,assassinée
8,slw,ditla,rhineston,burgmeist,fatale
9,fiancee,burgmeister,mixtecouleur,xwa,homme


In [8]:
term="muscat"

df_=[]
for column in columns:
    model = model_dic[column][0]
    token = stemmer.stem(term) if "clean_stem" in column else term
    mpow = model.predict_output_word([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])

print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : muscat


,clean CBOW,clean skip_gram,clean_stem CBOW,clean_stem skip_gram,online model
0,blanc,aoc,blanc,aoc,vins
1,cabernet,cuvee,bontem,cuve,vin
2,chateau,domaine,unek,chateau,raisin
3,muscat,appellation,steinhaus,cepag,muscat
4,alsace,vin,baudino,domain,cépage
5,merlot,chateau,tassin,vign,blanc
6,setti,millesime,soinn,vin,grains
7,trimoulet,pessac,biziou,muscat,jus
8,sauvignon,muscat,despringr,millesim,aoc
9,abbaye,vignes,vade,leognan,fromage


In [9]:
term="television"

df_=[]
for column in columns:
    model = model_dic[column][0]
    token = stemmer.stem(term) if "clean_stem" in column else term
    mpow = model.predict_output_word([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])

print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : television


,clean CBOW,clean skip_gram,clean_stem CBOW,clean_stem skip_gram,online model
0,television,television,televis,televis,television
1,numerique,televiseur,video,televiseur,network
2,audio,sources,cinem,hdmi,channel
3,sources,hdmi,surveil,connexion,entertainment
4,video,cinema,numer,cinem,awards
5,radio,hifi,moniteur,philip,tv
6,theatre,videos,hdmi,radio,disney
7,cinema,dvd,multimedi,multimedi,télévision
8,reception,tuner,vga,video,abc
9,images,reception,resolu,moniteur,radio


In [10]:
term="xbox"

df_=[]
for column in columns:
    model = model_dic[column][0]
    token = stemmer.stem(term) if data_type=="clean_stem" else term
    df_.append([k[0] for k in model.predict_output_word([token])])

print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : xbox


,clean CBOW,clean skip_gram,clean_stem CBOW,clean_stem skip_gram,online model
0,one,xbox,one,xbox,playstation
1,microsoft,playstation,microsoft,playstat,xbox
2,jeu,wii,wii,wii,nintendo
3,import,nintendo,playstat,nintendo,live
4,xbox,controller,xbox,controll,console
5,wii,psp,manet,jeux,wii
6,playstation,manette,nintendo,allemand,consoles
7,nintendo,remote,consol,kinect,sega
8,console,allemand,desir,peripher,gamecube
9,manette,microsoft,jeu,consol,arcade


## With mean

In [11]:
from multiprocessing import Pool


def get_matrix_features_means(X, dataset="train"):
    
    if dataset =="train":
        with Pool(4) as p:
            X_embedded_ = p.map(get_features_mean_train, X)
        X_embedded = np.vstack(X_embedded_)
    if dataset == "validation":
        with Pool(4) as p:
            X_embedded_ = p.map(get_features_mean_valid, X)
        X_embedded = np.vstack(X_embedded_)
    return X_embedded



In [12]:
for data_type, X_train, X_valid in [["clean", X_train_clean, X_valid_clean], ["clean_stem", X_train_clean_stem, X_valid_clean_stem]]:
    for sg in [0, 1]:
        for hs, negative in [[1,0], [0,10]]:
            
            print("Download Model")
            model_clean_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_model" %(Features_dimension, sg, hs, negative)
            model = gensim.models.Word2Vec.load(model_clean_dir)
            print("Model Downloaded")
            
            MODEL = model
            F_SIZE = Features_dimension

            def get_features_mean_train(lines):
                fm = np.mean([MODEL[x] for x  in lines],axis=0) 
                return fm


            def get_features_mean_valid(lines):
                    features = [MODEL[x] for x  in lines if x in MODEL]
                    if features == []:   
                        fm =np.ones(F_SIZE)
                    else :
                        fm = np.mean(features,axis=0)
                    return fm
        
            embedded_train_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_train_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_mean" %(Features_dimension, sg, hs, negative)
            embedded_valid_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_valid_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_mean" %(Features_dimension, sg, hs, negative)
            metadata_embedded_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_metadata_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_mean" %(Features_dimension, sg, hs, negative) 
            
            ts = time.time()
            X_embedded_train = get_matrix_features_means(X_train, "train")
            te = time.time()
            t_train = te-ts
            np.save(embedded_train_dir, X_embedded_train)
            print("Time conversion : %d seconds"%t_train)
            print("Shape Matrix : (%d,%d)"%X_embedded_train.shape)
                  
            ts = time.time()
            X_embedded_valid = get_matrix_features_means(X_valid, "validation")
            te = time.time()
            t_valid = te-ts
            np.save(embedded_valid_dir, X_embedded_valid)
            print("Time conversion : %d seconds"%t_valid)
            print("Shape Matrix : (%d,%d)"%X_embedded_valid.shape)
            
            metadata = {"t_train" : t_train, "t_valid" : t_valid, "train_dir" : embedded_train_dir, "valid_dir" : embedded_valid_dir,
                        "data_type": data_type, "hs": hs, "sg":sg, "negative":negative, "aggreg" : "mean"}
            pickle.dump(metadata, open(metadata_embedded_dir, "wb"))
              
        

Download Model
Model Downloaded


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Time conversion : 69 seconds
Shape Matrix : (900000,300)


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated 

Time conversion : 8 seconds
Shape Matrix : (100000,300)
Download Model
Model Downloaded


...

Time conversion : 7 seconds
Shape Matrix : (100000,300)
Download Model
Model Downloaded


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Time conversion : 72 seconds
Shape Matrix : (900000,300)


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated 

Time conversion : 8 seconds
Shape Matrix : (100000,300)


In [13]:
X_train = X_train_clean
X_valid = X_valid_clean

print("Download Model")
model = gensim.models.Word2Vec.load(model_online_dir)
print("Model Downloaded")

MODEL = model
F_SIZE = Features_dimension


def get_features_mean_valid(lines):
        features = [MODEL[x] for x  in lines if x in MODEL]
        if features == []:   
            fm =np.ones(F_SIZE)
        else :
            fm = np.mean(features,axis=0)
        return fm

embedded_train_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_train_Fdim_%d_mean" %(Features_dimension)
embedded_valid_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_valid_Fdim_%d_mean" %(Features_dimension)
metadata_embedded_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_metadata_Fdim_%d_mean" %(Features_dimension) 

ts = time.time()
X_embedded_train_ = get_matrix_features_means(X_train, "validation")
X_embedded_train = X_embedded_train_[X_embedded_train_.sum(1)!=300]

te = time.time()
t_train = te-ts
np.save(embedded_train_dir, X_embedded_train)
print("Time conversion : %d seconds"%t_train)
print("Shape Matrix : (%d,%d)"%X_embedded_train.shape)

ts = time.time()
X_embedded_valid = get_matrix_features_means(X_valid, "validation")
te = time.time()
t_valid = te-ts
np.save(embedded_valid_dir, X_embedded_valid)
print("Time conversion : %d seconds"%t_valid)
print("Shape Matrix : (%d,%d)"%X_embedded_valid.shape)

metadata = {"t_train" : t_train, "t_valid" : t_valid, "train_dir" : embedded_train_dir, "valid_dir" : embedded_valid_dir,
                        "data_type": data_type, "hs": 0, "sg":0, "negative":5, "aggreg" : "mean"}
pickle.dump(metadata, open(metadata_embedded_dir, "wb"))



Download Model
Model Downloaded


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated 

Time conversion : 75 seconds
Shape Matrix : (895815,300)


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated 

Time conversion : 6 seconds
Shape Matrix : (100000,300)


## With TfIdf

Learn TFIDF weights. Configure Vectorizer to read list of list of token.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

def dummy_fun(doc):
    return doc

tfidf_clean = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None) 

## ts = time.time()
tfidf_clean.fit(X_train_clean)
vocToIdf_clean = dict(zip([k[0] for k in sorted(tfidf_clean.vocabulary_.items(), key = lambda x : x[1])],tfidf_clean.idf_,))
features_list_clean = tfidf_clean.get_feature_names()
te = time.time()
print("Time Building weight and dictionnary : %d seconds"%(te-ts))


Time Building weight and dictionnary : 21 seconds


In [15]:
tfidf_clean_stem = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None) 

## ts = time.time()
tfidf_clean_stem.fit(X_train_clean_stem)
vocToIdf_clean_stem = dict(zip([k[0] for k in sorted(tfidf_clean_stem.vocabulary_.items(), key = lambda x : x[1])],tfidf_clean_stem.idf_,))
features_list_clean_stem = tfidf_clean_stem.get_feature_names()
te = time.time()
print("Time Building weight and dictionnary : %d seconds"%(te-ts))


Time Building weight and dictionnary : 34 seconds


In [16]:
def get_matrix_features_idf(X, dataset="train"):
    if dataset =="train":
        with Pool(4) as p:
            X_embedded_ = p.map(get_features_idf_train, X)
        X_embedded = np.vstack(X_embedded_)
    if dataset == "validation":
        with Pool(4) as p:
            X_embedded_ = p.map(get_features_idf_valid, X)
        X_embedded = np.vstack(X_embedded_)
    return X_embedded


In [17]:
for data_type, X_train, X_valid, vocToIdf in [["clean", X_train_clean, X_valid_clean, vocToIdf_clean], ["clean_stem", X_train_clean_stem, X_valid_clean_stem, vocToIdf_clean_stem]]:
    for sg in [0,1]:
        for hs, negative in [[1,0], [0,10]]:
            
            print("Download Model")
            model_clean_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_model" %(Features_dimension, sg, hs, negative)
            model = gensim.models.Word2Vec.load(model_clean_dir)
            print("Model Downloaded")
            
            MODEL = model
            IDF_DIC = vocToIdf
            F_SIZE = Features_dimension

            def get_features_idf_train(lines):
                fm = np.mean([MODEL[x] * IDF_DIC[x] for x  in lines],axis=0) 
                return fm


            def get_features_idf_valid(lines):
                    features = [MODEL[x] * IDF_DIC[x] for x  in lines if x in MODEL ]
                    if features == []:   
                        fm =np.ones(F_SIZE)
                    else :
                        fm = np.mean(features,axis=0)
                    return fm
                
            embedded_train_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_train_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_tfidf" %(Features_dimension, sg, hs, negative)
            embedded_valid_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_valid_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_tfidf" %(Features_dimension, sg, hs, negative)
            metadata_embedded_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_metadata_min_count_1_Fdim_%d_sg_%d_hs_%d_negative_%d_tfidf" %(Features_dimension, sg, hs, negative)

            
            ts = time.time()
            X_embedded_train = get_matrix_features_idf(X_train, "train")
            te = time.time()
            t_train = te-ts
            np.save(embedded_train_dir, X_embedded_train)
            print("Time conversion : %d seconds"%t_train)
            print("Shape Matrix : (%d,%d)"%X_embedded_train.shape)
                  
            ts = time.time()
            X_embedded_valid = get_matrix_features_idf(X_valid, "validation")
            te = time.time()
            t_valid = te-ts
            np.save(embedded_valid_dir, X_embedded_valid)
            print("Time conversion : %d seconds"%t_valid)
            print("Shape Matrix : (%d,%d)"%X_embedded_valid.shape)
            
            metadata = {"t_train" : t_train, "t_valid" : t_valid, "train_dir" : embedded_train_dir, "valid_dir" : embedded_valid_dir,
                        "data_type": data_type, "hs": hs, "sg":sg, "negative":negative, "aggreg" : "tfidf"}
            pickle.dump(metadata, open(metadata_embedded_dir, "wb"))
              


                

Download Model
Model Downloaded


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Time conversion : 86 seconds
Shape Matrix : (900000,300)


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated 

Time conversion : 10 seconds
Shape Matrix : (100000,300)
Download Model
Model Downloaded


...

Time conversion : 9 seconds
Shape Matrix : (100000,300)
Download Model
Model Downloaded


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Time conversion : 62 seconds
Shape Matrix : (900000,300)


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated 

Time conversion : 8 seconds
Shape Matrix : (100000,300)


In [21]:
print("Download Model")
model = gensim.models.Word2Vec.load(model_online_dir)
print("Model Downloaded")

X_train = X_train_clean
X_valid = X_valid_clean

MODEL = model
IDF_DIC = vocToIdf_clean
F_SIZE = Features_dimension

def get_features_idf_valid(lines):
        features = [MODEL[x] * IDF_DIC[x] for x  in lines if x in MODEL and x in IDF_DIC ]
        if features == []:   
            fm =np.ones(F_SIZE)
        else :
            fm = np.mean(features,axis=0)
        return fm

embedded_train_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_train_Fdim_%d_train_tfidf" %(Features_dimension)
embedded_valid_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_valid_Fdim_%d_valid_tfidf" %(Features_dimension)
metadata_embedded_dir = DATA_WORD2VEC_DIR+"_"+data_type+"_Embedded_metadata_Fdim_%d_metadata_tfidf" %(Features_dimension)


ts = time.time()
X_embedded_train_ = get_matrix_features_idf(X_train, "validation")
X_embedded_train = X_embedded_train_[X_embedded_train_.sum(1)!=300]
te = time.time()
t_train = te-ts
np.save(embedded_train_dir, X_embedded_train)
print("Time conversion : %d seconds"%t_train)
print("Shape Matrix : (%d,%d)"%X_embedded_train.shape)

ts = time.time()
X_embedded_valid = get_matrix_features_idf(X_valid, "validation")
te = time.time()
t_valid = te-ts
np.save(embedded_valid_dir, X_embedded_valid)
print("Time conversion : %d seconds"%t_valid)
print("Shape Matrix : (%d,%d)"%X_embedded_valid.shape)

metadata = {"t_train" : t_train, "t_valid" : t_valid, "train_dir" : embedded_train_dir, "valid_dir" : embedded_valid_dir,
                        "data_type": "clean", "hs":0 , "sg":0, "negative":5, "aggreg" : "tfidf"}
pickle.dump(metadata, open(metadata_embedded_dir, "wb"))

Download Model
Model Downloaded


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated 

Time conversion : 101 seconds
Shape Matrix : (895815,300)


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Call to deprecate

Time conversion : 10 seconds
Shape Matrix : (100000,300)


# Tsne

In [ ]:
from sklearn.manifold import TSNE
n_words=200

#words = [k for k in model.wv.vocab.keys()][100:100+n_words]
words = [k[0] for k in sorted(C.items(), key = lambda x : x[1], reverse=True)[:n_words]]
X = model[words]
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

In [ ]:
import matplotlib.pyplot as plt
fig =plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1)
ax.plot(X_tsne[:,0], X_tsne[:,1], linestyle="None", marker=".")
for i in range(n_words):
    ax.text(X_tsne[i,0], X_tsne[i,1],  words[i], fontsize=10)
plt.show()